import time
plc=1
time.sleep(3000*plc)         #run after 1 houre = 3600 seconds

In [1]:
import os
import glob
import scipy
import random
import bisect
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import signal
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
from preprocess import *                         #ماژول استخراج پنجره ها
from data import *                          #ماژول محلی ورود داده ها
from augment import *                            #ماژول های داده افزایی
from lstm_cnn import *                           #ماژول های داده افزایی با lstm_cnn
import augment
import importlib
importlib.reload(augment)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report,recall_score,precision_score
from tensorflow.keras import callbacks
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D,Conv1D,Dropout,MaxPooling1D,MaxPooling2D,Flatten,Dense
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
cwd = os.getcwd() #
fullscrn()

<h1><center><div style="direction:rtl;font-family:B Nazanin">Importing Data</div></center></h1>

In [2]:
rate=1460
cls_num=10
btch=80
for i in range (0,cls_num):
    vars()['ecg'+str(i)],vars()['ecg_tst'+str(i)]=ACSF1(i)

os.chdir(cwd)
clses_lens=np.array([])
i=0                               #جمع آوری داده ها و چاپ تعداد نمونه ی هر کلاس
ecg=np.array(ecg1)
print(1,len(vars()['ecg'+str(1)]), end='\t')
clses_lens=np.append(clses_lens,len(vars()['ecg'+str(1)]))
for i in range (1,cls_num):
    ecg=np.concatenate((ecg,vars()['ecg'+str(i)]),axis=0)
    clses_lens=np.append(clses_lens,len(vars()['ecg'+str(i)]))
    print(i,len(vars()['ecg'+str(i)]), end='\t')

mx_sig=max(clses_lens)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


first_train_shape= (100, 1461)
first_test_shape= (100, 1461)
classes_quantity= 10
tr_lbls=	 {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}
Count_labels= [10 10 10 10 10 10 10 10 10]
max(train_feature_Altitude)= 12.42995
min(train_feature_Altitude)= -1.352876
first_train_sample=
 [ 9.         -0.58475375 -0.58475375 ... -0.5786034   1.7327257
 -0.58473404]
1 10	1 10	2 10	3 10	4 10	5 10	6 10	7 10	8 10	9 10	

In [3]:
np.shape(ecg)

(100, 1461)

<h2><center><div style="direction:rtl;font-family:B Nazanin">Base Train windows</div></center></h1>

In [4]:
smpl_rte=rate                                              # در ماژول ها نیز همین مقدار ثبت شده
i=0
windws=np.array([])

for cls in range (0,cls_num):                                 #ساخت پنجره های داده های آموزش اصلی
    vars()['wndws'+str(cls)]=np.array(vars()['ecg'+str(cls)])  
    
print("each class and its windows =")

cls_wndws=np.array([])
for cls in range (0,cls_num):                                # آرایه ی تعداد پنجره ی هر کلاس
    wns=len(vars()['wndws'+str(cls)])
    cls_wndws=np.append(cls_wndws,wns)
    print(cls, wns, end='\t')

mx_wndws=int(np.max(cls_wndws))                        # حداکثر تعداد پنجره ی موجود بین کلاس ها
print('\n max =', mx_wndws)

each class and its windows =
0 10	1 10	2 10	3 10	4 10	5 10	6 10	7 10	8 10	9 10	
 max = 10


<h1><center><div style="direction:rtl;font-family:B Nazanin">Data Augmentation</div></center></h1>

In [5]:
smpl_rte=len(wndws1[0])                                        # در ماژول ها نیز همین مقدار ثبت شده

#add augmented data to base data
xtrain=np.empty((0,len(wndws1[0])), float) 
for i in range (0,cls_num):   
    xtrain=np.append(xtrain,vars()['wndws'+str(i)],axis=0)
    print('Up to class ', i, 'train shape = ', np.shape(xtrain))

Up to class  0 train shape =  (10, 1461)
Up to class  1 train shape =  (20, 1461)
Up to class  2 train shape =  (30, 1461)
Up to class  3 train shape =  (40, 1461)
Up to class  4 train shape =  (50, 1461)
Up to class  5 train shape =  (60, 1461)
Up to class  6 train shape =  (70, 1461)
Up to class  7 train shape =  (80, 1461)
Up to class  8 train shape =  (90, 1461)
Up to class  9 train shape =  (100, 1461)


In [6]:
print(np.bincount(np.int16(xtrain[:,-1])))    #تعداد پنجره در هر کلاس از 0 تا 17

[10 10 10 10 10 10 10 10 10 10]


In [7]:
xtrain[:,-1]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 5.,
       5., 5., 5., 5., 5., 5., 5., 5., 5., 6., 6., 6., 6., 6., 6., 6., 6.,
       6., 6., 7., 7., 7., 7., 7., 7., 7., 7., 7., 7., 8., 8., 8., 8., 8.,
       8., 8., 8., 8., 8., 9., 9., 9., 9., 9., 9., 9., 9., 9., 9.])

In [8]:
mx_aug=np.max(xtrain[:,:-1])
mx=np.max(ecg[:,:-1])
mn_aug=np.min(xtrain[:,:-1])
mn=np.min(ecg[:,:-1])

#for i in range (0,cls_num):             #نرمالسازی داده های افزایشی
xtrain[:,:-1]= 2*(xtrain[:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1

In [9]:
print('min trn =', np.min(xtrain[:,:-1]))
print('max trn =', np.max(xtrain[:,:-1]))

min trn = -1.0
max trn = 1.0


<h3><center><div style="direction:rtl;font-family:B Nazanin">Test Windows</div></center></h1>

In [10]:
wndws_test=np.empty((0,len(wndws1[0])), float)
i=0
windws=np.array([])
for cls in range (0,cls_num):                                 #ساخت پنجره های داده های آموزش اصلی
    vars()['wndws_tst'+str(cls)]=np.empty((0,len(wndws1[0])), float)
    vars()['wndws_tst'+str(cls)]=np.append(vars()['wndws_tst'+str(cls)],vars()['ecg_tst'+str(cls)],axis=0)   

In [11]:
#print(np.max(wndws_tst))
for i in range (0,cls_num):                # Normalization test windows
    cls=i
    print('\n cls', i, ' >> ')
    print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
    print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))

    vars()['wndws_tst'+str(i)][:,:-1]= 2*(vars()['wndws_tst'+str(i)][:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1
    
    print('after normalizing >>')
    print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
    print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))
    
print("each class and its windows =")                    #تجمیع کلاس های تست
cls_wndws=np.array([])
for cls in range (0,cls_num):                                # آرایه ی تعداد پنجره ی هر کلاس
    wns=len(vars()['wndws_tst'+str(cls)])
    cls_wndws=np.append(cls_wndws,wns)
    print(cls, wns, end='\t')
    wndws_test=np.append(wndws_test,vars()['wndws_tst'+str(cls)],axis=0)
    
mx_wndws=int(np.max(cls_wndws))                        # حداکثر تعداد پنجره ی موجود بین کلاس ها
print('\n max instance in classes of test windows =', mx_wndws)


 cls 0  >> 
max magnitude class 0  =  7.793805500000001
min magnitude class 0  =  -0.9384364300000007
after normalizing >>
max magnitude class 0  =  0.3272577771786427
min magnitude class 0  =  -0.9398614522159678

 cls 1  >> 
max magnitude class 1  =  1.7608700000000006
min magnitude class 1  =  -1.2472862
after normalizing >>
max magnitude class 1  =  -0.548170164812354
min magnitude class 1  =  -0.9846780623944609

 cls 2  >> 
max magnitude class 2  =  1.7483563000000004
min magnitude class 2  =  -0.9271888500000005
after normalizing >>
max magnitude class 2  =  -0.5499860043216098
min magnitude class 2  =  -0.9382293370024405

 cls 3  >> 
max magnitude class 3  =  11.340412000000004
min magnitude class 3  =  -0.89856476
after normalizing >>
max magnitude class 3  =  0.8418991867125079
min magnitude class 3  =  -0.9340757490517545

 cls 4  >> 
max magnitude class 4  =  1.6795779999999993
min magnitude class 4  =  -1.1284813
after normalizing >>
max magnitude class 4  =  -0.55996629

In [12]:
print('min tst =', np.min(wndws_test[:,:-1]))
print('max tst =', np.max(wndws_test[:,:-1]))

min tst = -1.003488762028919
max tst = 0.8418991867125079


In [13]:
len(wndws_test)

100

In [14]:
np.shape(wndws_test)

(100, 1461)

In [15]:
Xtest=wndws_test[:,:-1]
ytest=np.int16(wndws_test[:,-1])

In [16]:
fullprint(ytest)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8,
       8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9], dtype=int16)


<h3><center><div style="direction:rtl;font-family:B Nazanin">Train Windows</div></center></h1>

In [17]:
#ytraint=np.int32(np.array([ytrain]).T)
#trainx=np.concatenate((Xtrain, ytraint), axis=1)
trainx=np.random.permutation(xtrain)
Xtrain=np.array(trainx[:,:-1])
ytrain=np.int16(trainx[:,-1])
'''
ytestt=np.int32(np.array([ytest]).T)
testx=np.concatenate((Xtest, ytestt), axis=1)
testx=np.random.permutation(testx)
Xtest=np.int32(testx[:,:-1])
ytest=np.int32(testx[:,-1])'''

'\nytestt=np.int32(np.array([ytest]).T)\ntestx=np.concatenate((Xtest, ytestt), axis=1)\ntestx=np.random.permutation(testx)\nXtest=np.int32(testx[:,:-1])\nytest=np.int32(testx[:,-1])'

In [18]:
np.shape(wndws1)

(10, 1461)

<h3><center><div style="direction:rtl;font-family:B Nazanin">Validation Windows</div></center></h1>

In [19]:
Xvalid=np.array(Xtrain)
yvalid=np.array(ytrain)

<h1><center><div style="direction:rtl;font-family:B Nazanin">CNN And UnBalanced RAW Data</div></center></h1>
<h1><center><div style="direction:rtl;font-family:Arial">Cross Entropy Loss Function</div></center></h1>

In [20]:
X_train=np.array(Xtrain)
y_train=np.array(ytrain)
X_test=np.array(Xtest)
y_test=np.array(ytest)
X_valid=np.array(Xtrain)
y_valid=np.array(ytrain)
print('X_train => ', X_train.shape)
print('y_train => ', y_train.shape)
print('X_test  => ', X_test.shape)
print('y_test  => ', y_test.shape)
print('X_valid  => ', X_valid.shape)
print('y_valid  => ', y_valid.shape)

X_train =>  (100, 1460)
y_train =>  (100,)
X_test  =>  (100, 1460)
y_test  =>  (100,)
X_valid  =>  (100, 1460)
y_valid  =>  (100,)


In [21]:
y_train=np.array(ytrain)
np.min(y_train)

0

In [22]:
y_train_cat=to_categorical(y_train)
y_valid_cat=to_categorical(y_valid)
y_test_cat=to_categorical(y_test)


X_train = np.expand_dims(X_train, axis=2)
X_valid = np.expand_dims(X_valid, axis=2)
X_test = np.expand_dims(X_test, axis=2)
#y_train=np.transpose([y_train])

In [23]:
accuracy=0
acc_crs=np.array([])
##for i in range (10):
##    if accuracy<0.2 :

verbose, epochs, batch_size = 1, 1000, btch
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train_cat.shape[1]
steps_per_epoch = len(X_train)//batch_size
validation_steps = len(X_valid)//batch_size # if you have test data

model_crs = Sequential()
BatchNormalization()
model_crs.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
#model_crs.add(MaxPooling1D(pool_size=3))
BatchNormalization()
model_crs.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
#model_crs.add(Dropout(0.5))
model_crs.add(MaxPooling1D(pool_size=3))
BatchNormalization()
model_crs.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
BatchNormalization()
model_crs.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
BatchNormalization()
model_crs.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
'''model_crs.add(Conv1D(filters=512, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
model_crs.add(Conv1D(filters=1024, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
'''
model_crs.add(Flatten())
#model_crs.add(Dropout(0.5))
model_crs.add(Dense(2000, activation='relu'))
model_crs.add(Dense(n_outputs, activation='softmax'))
model_crs.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
# fit network   #CategoricalCrossentropy #sparse_categorical_crossentropy #SparseCategoricalCrossentropy

earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", mode ="max", patience = 20, restore_best_weights = True)

model_crs.fit(X_train, y_train_cat, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data = (X_valid, y_valid_cat), callbacks =[earlystopping])
# evaluate model_crs
_, accuracy = model_crs.evaluate(X_valid, y_valid_cat, batch_size=batch_size, verbose=1)

print('np.shape(X_test)=',np.shape(X_test))


model_crs.summary()
print('Accuracy= ', accuracy)

#output = K.function([model_crs.layers[0].input],[model_crs.layers[3].output])
#output(X_test)[0]
#y_pred= model_crs.predict(X_test)                # Function 1

##    print('Round ', i, '==>', 'accuracy = ', accuracy)
##    acc_crs=np.append(acc_crs,accuracy)
#y_pred_crs = model_crs.predict_classes(X_test)   # Function 2

Epoch 1/1000
2/2 [==============================] - 4s 453ms/step - loss: 2.5318 - accuracy: 0.1000 - val_loss: 2.3203 - val_accuracy: 0.1000
Epoch 2/1000
2/2 [==============================] - 0s 34ms/step - loss: 2.3172 - accuracy: 0.1000 - val_loss: 2.3066 - val_accuracy: 0.1000
Epoch 3/1000
2/2 [==============================] - 0s 54ms/step - loss: 2.3106 - accuracy: 0.1000 - val_loss: 2.2998 - val_accuracy: 0.1600
Epoch 4/1000
2/2 [==============================] - 0s 60ms/step - loss: 2.3043 - accuracy: 0.1200 - val_loss: 2.2925 - val_accuracy: 0.1900
Epoch 5/1000
2/2 [==============================] - 0s 35ms/step - loss: 2.2882 - accuracy: 0.1800 - val_loss: 2.9263 - val_accuracy: 0.1000
Epoch 6/1000
2/2 [==============================] - 0s 34ms/step - loss: 2.8110 - accuracy: 0.1100 - val_loss: 2.2769 - val_accuracy: 0.1700
Epoch 7/1000
2/2 [==============================] - 0s 35ms/step - loss: 2.2722 - accuracy: 0.1500 - val_loss: 2.2316 - val_accuracy: 0.1000
Epoch 8/1000

2/2 [==============================] - 0s 4ms/step - loss: 1.4712 - accuracy: 0.4700
np.shape(X_test)= (100, 1460, 1)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 1458, 32)          128       
                                                                 
 conv1d_1 (Conv1D)           (None, 1456, 64)          6208      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 485, 64)          0         
 )                                                               
                                                                 
 conv1d_2 (Conv1D)           (None, 483, 128)          24704     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 161, 128)         0         
 1D)                                                             
    

In [24]:
def maxindx(ar):
    ar_cpy=np.array(ar)
    pred=np.argmax(ar,axis=1)
    for i in range (len(pred)):
        if pred[i]==0:
            ar_cpy[i,0]=-1000
    pred=np.argmax(ar_cpy,axis=1)
    return(pred)

In [25]:
predict_x=model_crs.predict(X_test)              # Function 1

y_pred_crs=maxindx(predict_x)                    # function from augment.py to remove 0 index predictions

#y_pred_crs = model_crs.predict_classes(X_test)

cm = confusion_matrix(y_test, y_pred_crs)
fullprint(cm)
acc3=accuracy_score(y_test, y_pred_crs)

print(classification_report(y_test, y_pred_crs))

#sns.heatmap(cm, annot=True)

#plt.imshow(cm)

4/4 [==============================] - 0s 21ms/step
array([[ 0,  0,  0,  1,  0,  2,  6,  0,  0,  1],
       [ 0,  0,  0,  0,  9,  1,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  6,  4,  0,  0,  0,  0],
       [ 0,  0,  0,  6,  0,  0,  0,  2,  2,  0],
       [ 0,  0,  0,  0, 10,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  1,  4,  5,  0,  0,  0],
       [ 0,  0,  0,  0,  1,  1,  8,  0,  0,  0],
       [ 0,  0,  0,  1,  0,  0,  4,  2,  3,  0],
       [ 0,  0,  0,  4,  0,  0,  1,  3,  2,  0],
       [ 0,  0,  0,  0,  0,  0, 10,  0,  0,  0]], dtype=int64)
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        10
           1       0.00      0.00      0.00        10
           2       0.00      0.00      0.00        10
           3       0.50      0.60      0.55        10
           4       0.37      1.00      0.54        10
           5       0.33      0.40      0.36        10
           6       0.24      0.80      0.36        10
           

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [26]:
rpt_cnn_clsfctn=5
rslt=np.array([])
rslts=np.array([])
xtrain=np.empty((0,len(wndws1[0])), float) 
rprt=np.empty((0,3),float)


for i in range (1,cls_num+1):                           #تعریف آرایه ی پنجره های کلاس ها
    vars()['ecg_aug10'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug11'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug12'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug20'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug21'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug22'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug30'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug31'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug32'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug40'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug41'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug42'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug50'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug51'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug52'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 

for i in range (1,cls_num+1):                                     #تعریف آرایه ی پنجره های کلاس ها
    vlum_win=mx_wndws-len(vars()['wndws'+str(i)])         #میزان داده افزایی در روش های عمیق (پنجره)
    #print("vlum_win=",vlum_win)
    vars()['ecg_aug10'+str(i)]=np.append(vars()['ecg_aug10'+str(i)],sclRaw(ecg,i,vlum_win,mag=.05),axis=0)
    vars()['ecg_aug11'+str(i)]=np.append(vars()['ecg_aug11'+str(i)],sclRaw(ecg,i,vlum_win,mag=.1),axis=0)
    vars()['ecg_aug12'+str(i)]=np.append(vars()['ecg_aug12'+str(i)],sclRaw(ecg,i,vlum_win,mag=.2),axis=0)
    vars()['ecg_aug20'+str(i)]=np.append(vars()['ecg_aug20'+str(i)],MgWrpRaw(ecg,i,vlum_win,mag=.05),axis=0)
    vars()['ecg_aug21'+str(i)]=np.append(vars()['ecg_aug21'+str(i)],MgWrpRaw(ecg,i,vlum_win,mag=.1),axis=0)
    vars()['ecg_aug22'+str(i)]=np.append(vars()['ecg_aug22'+str(i)],MgWrpRaw(ecg,i,vlum_win,mag=.2),axis=0)
    vars()['ecg_aug30'+str(i)]=np.append(vars()['ecg_aug30'+str(i)],TimWrpRaw(ecg,i,vlum_win,mag=.05),axis=0)
    vars()['ecg_aug31'+str(i)]=np.append(vars()['ecg_aug31'+str(i)],TimWrpRaw(ecg,i,vlum_win,mag=.1),axis=0)
    vars()['ecg_aug32'+str(i)]=np.append(vars()['ecg_aug32'+str(i)],TimWrpRaw(ecg,i,vlum_win,mag=.2),axis=0)
    vars()['ecg_aug40'+str(i)]=np.append(vars()['ecg_aug40'+str(i)],GDORaw(ecg,i,vlum_win,mag=.05),axis=0)
    vars()['ecg_aug41'+str(i)]=np.append(vars()['ecg_aug41'+str(i)],GDORaw(ecg,i,vlum_win,mag=.1),axis=0)
    vars()['ecg_aug42'+str(i)]=np.append(vars()['ecg_aug42'+str(i)],GDORaw(ecg,i,vlum_win,mag=.2),axis=0)
    vars()['ecg_aug50'+str(i)]=np.append(vars()['ecg_aug50'+str(i)],frqncRaw(furir_windws,i,vlum_win,mag=.05),axis=0)
    vars()['ecg_aug51'+str(i)]=np.append(vars()['ecg_aug51'+str(i)],frqncRaw(furir_windws,i,vlum_win,mag=.1),axis=0)
    vars()['ecg_aug52'+str(i)]=np.append(vars()['ecg_aug52'+str(i)],frqncRaw(furir_windws,i,vlum_win,mag=.2),axis=0)    

for methd in range(1,6):
    for Mgntud in range(0,3):
        xtrain=np.empty((0,len(wndws1[0])), float) 
        rprt=np.empty((0,3),float)
        for repeat in range(1,rpt_cnn_clsfctn+1):
            for i in range (1,cls_num+1):   
                xtrain=np.append(xtrain,vars()['wndws'+str(i)],axis=0)
                xtrain=np.append(xtrain,vars()['ecg_aug'+str(methd)+str(Mgntud)+str(i)],axis=0)
                #print('Up to class ', i, 'train shape = ', np.shape(xtrain))

            # Normalization train windows
            mx_aug=np.max(xtrain[:,:-1])
            mx=np.max(ecg[:,:-1])
            mn_aug=np.min(xtrain[:,:-1])
            mn=np.min(ecg[:,:-1])
            xtrain[:,:-1]= 2*(xtrain[:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1


            ###Test Windows:
            wndws_test=np.empty((0,len(wndws1[0])), float)
            i=0
            windws=np.array([])
            for cls in range (1,cls_num+1):                                 #ساخت پنجره های داده های آموزش اصلی
                vars()['wndws_tst'+str(cls)]=np.empty((0,len(wndws1[0])), float)
                vars()['wndws_tst'+str(cls)]=np.append(vars()['wndws_tst'+str(cls)],vars()['ecg_tst'+str(cls)],axis=0)   

            # Normalization test windows
            for i in range (1,cls_num+1):                
                cls=i
                #print('\n cls', i, ' >> ')
                #print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
                #print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))
                vars()['wndws_tst'+str(i)][:,:-1]= 2*(vars()['wndws_tst'+str(i)][:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1
                #print('after normalizing >>')
                #print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
                #print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))
            #print("each class and its windows =")                    #تجمیع کلاس های تست
            cls_wndws=np.array([])
            for cls in range (1,cls_num+1):                                # آرایه ی تعداد پنجره ی هر کلاس
                wns=len(vars()['wndws_tst'+str(cls)])
                cls_wndws=np.append(cls_wndws,wns)
                #print(cls, wns, end='\t')
                wndws_test=np.append(wndws_test,vars()['wndws_tst'+str(cls)],axis=0)
            #mx_wndws=int(np.max(cls_wndws))                        # حداکثر تعداد پنجره ی موجود بین کلاس ها
            #print('\n max instance in classes of test windows =', mx_wndws)

            #print('min tst =', np.min(wndws_test[:,:-1]))
            #print('max tst =', np.max(wndws_test[:,:-1]))

            Xtest=wndws_test[:,:-1]
            ytest=np.int16(wndws_test[:,-1])

            #fullprint(ytest)

            trainx=np.random.permutation(xtrain)
            Xtrain=np.array(trainx[:,:-1])
            ytrain=np.int16(trainx[:,-1])

            X_train=np.array(Xtrain)
            y_train=np.array(ytrain)
            X_test=np.array(Xtest)
            y_test=np.array(ytest)
            X_valid=np.array(Xtrain)
            y_valid=np.array(ytrain)
            #print('X_train => ', X_train.shape)
            #print('y_train => ', y_train.shape)
            #print('X_test  => ', X_test.shape)
            #print('y_test  => ', y_test.shape)
            #print('X_valid  => ', X_valid.shape)
            #print('y_valid  => ', y_valid.shape)

            y_train_cat=to_categorical(y_train)
            y_valid_cat=to_categorical(y_valid)
            y_test_cat=to_categorical(y_test)
            X_train = np.expand_dims(X_train, axis=2)
            X_valid = np.expand_dims(X_valid, axis=2)
            X_test = np.expand_dims(X_test, axis=2)

            ####### NETWORK #######
            #Yildirin

            accuracy=0
            acc_crs=np.array([])
            ##for i in range (10):
            ##    if accuracy<0.2 :

            verbose, epochs, batch_size = 0, 1000, btch
            n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train_cat.shape[1]
            steps_per_epoch = len(X_train)//batch_size
            validation_steps = len(X_valid)//batch_size # if you have test data



            model_crs = Sequential()
            #model_crs.add(Conv1D(input_shape=x_train.shape[1:],filters=64,kernel_size=9,padding="same", activation="relu"))
            model_crs.add(Conv1D(filters=128, kernel_size=12, strides=1, activation='relu', input_shape=(n_timesteps,n_features))) #Replaced
            model_crs.add(BatchNormalization())
            model_crs.add(MaxPooling1D(pool_size=2,strides=3))

            model_crs.add(Conv1D(filters=32, kernel_size=7, strides=1, padding="same", activation="relu"))
            model_crs.add(BatchNormalization())
            model_crs.add(MaxPooling1D(pool_size=2,strides=2))

            model_crs.add(Conv1D(filters=32, kernel_size=10, strides=1, padding="same", activation="relu"))
            model_crs.add(Conv1D(filters=128, kernel_size=5, strides=2, padding="same", activation="relu"))
            model_crs.add(MaxPooling1D(pool_size=2,strides=2))

            model_crs.add(Conv1D(filters=256, kernel_size=15, strides=1, padding="same", activation="relu"))
            model_crs.add(MaxPooling1D(pool_size=2,strides=2))

            model_crs.add(Conv1D(filters=512, kernel_size=5, strides=1, padding="same", activation="relu"))
            #model_crs.add(Conv1D(filters=128, kernel_size=3, strides=1, padding="same", activation="relu"))


            model_crs.add(Flatten())
            model_crs.add(Dropout(0.1))
            model_crs.add(Dense(units=512,activation="relu"))
            #model_crs.add(Dense(units=2048,activation="relu"))
            model_crs.add(Dense(n_outputs, activation='softmax'))   #Replaced here from old Model

            model_crs.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
            # fit network   #CategoricalCrossentropy #sparse_categorical_crossentropy #SparseCategoricalCrossentropy

            earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", mode ="max", patience = 50, restore_best_weights = True)

            model_crs.fit(X_train, y_train_cat, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data = (X_valid, y_valid_cat), callbacks =[earlystopping])

            _, accuracy = model_crs.evaluate(X_valid, y_valid_cat, batch_size=batch_size, verbose=verbose)


            predict_x=model_crs.predict(X_test)              # Function 1
            y_pred_crs=maxindx(predict_x)                    # function from augment.py to remove 0 index predictions

            rprt0=classification_report(y_test, y_pred_crs,output_dict=True)
            rprt_row=np.array([])
            rprt_row=np.append(rprt_row,rprt0['accuracy'])
            rprt_row=np.append(rprt_row,rprt0['macro avg']['f1-score'])
            rprt_row=np.append(rprt_row,rprt0['weighted avg']['f1-score'])
            rprt=np.append(rprt,np.array([rprt_row]),axis=0)
        maxm=rprt[np.argmax(rprt[:,1])]
        avrg=np.mean(rprt,axis=0)
        rslts=np.append(rslts,['mthd '+str(methd)+'  Mgntud '+str(Mgntud)+'  average= '+str(np.int16(100*avrg))+'  max= '+str(np.int16(100*maxm))+'  var= '+str(np.int16(np.var(100*rprt[:,1])))],axis=0)

Epoch 1/1000
2/2 [==============================] - 1s 354ms/step - loss: 4.2569 - accuracy: 0.0900 - val_loss: 2.3594 - val_accuracy: 0.1000
Epoch 2/1000
2/2 [==============================] - 0s 46ms/step - loss: 2.3353 - accuracy: 0.1000 - val_loss: 2.3084 - val_accuracy: 0.1000
Epoch 3/1000
2/2 [==============================] - 0s 46ms/step - loss: 2.3131 - accuracy: 0.0900 - val_loss: 2.3055 - val_accuracy: 0.1000
Epoch 4/1000
2/2 [==============================] - 0s 50ms/step - loss: 2.3083 - accuracy: 0.0700 - val_loss: 2.3008 - val_accuracy: 0.1000
Epoch 5/1000
2/2 [==============================] - 0s 95ms/step - loss: 2.3097 - accuracy: 0.0800 - val_loss: 2.2999 - val_accuracy: 0.1800
Epoch 6/1000
2/2 [==============================] - 0s 94ms/step - loss: 2.3060 - accuracy: 0.1400 - val_loss: 2.2964 - val_accuracy: 0.2300
Epoch 7/1000
2/2 [==============================] - 0s 47ms/step - loss: 2.3037 - accuracy: 0.2000 - val_loss: 2.2678 - val_accuracy: 0.2000
Epoch 8/1000

 max_pooling1d_7 (MaxPooling  (None, 17, 512)          0         
 1D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 8704)              0         
                                                                 
 dense_2 (Dense)             (None, 2000)              17410000  
                                                                 
 dense_3 (Dense)             (None, 10)                20010     
                                                                 
Total params: 18,144,122
Trainable params: 18,144,122
Non-trainable params: 0
_________________________________________________________________
Accuracy=  0.4099999964237213


In [27]:
predict_x=model_crs.predict(X_test)              # Function 1

y_pred_crs=maxindx(predict_x)                    # function from augment.py to remove 0 index predictions

#y_pred_crs = model_crs.predict_classes(X_test)

cm = confusion_matrix(y_test, y_pred_crs)
fullprint(cm)
acc3=accuracy_score(y_test, y_pred_crs)

print(classification_report(y_test, y_pred_crs))

#sns.heatmap(cm, annot=True)

#plt.imshow(cm)

4/4 [==============================] - 0s 17ms/step
array([[ 0,  3,  0,  1,  0,  6,  0,  0,  0,  0],
       [ 0,  9,  0,  0,  0,  1,  0,  0,  0,  0],
       [ 0,  7,  0,  0,  0,  3,  0,  0,  0,  0],
       [ 0,  0,  0,  6,  0,  0,  0,  2,  2,  0],
       [ 0, 10,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  2,  0,  0,  0,  8,  0,  0,  0,  0],
       [ 0,  1,  0,  0,  0,  9,  0,  0,  0,  0],
       [ 0,  0,  0,  2,  0,  4,  0,  3,  1,  0],
       [ 0,  1,  0,  3,  0,  0,  0,  4,  2,  0],
       [ 0,  0,  0,  0,  0, 10,  0,  0,  0,  0]], dtype=int64)
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        10
           1       0.27      0.90      0.42        10
           2       0.00      0.00      0.00        10
           3       0.50      0.60      0.55        10
           4       0.00      0.00      0.00        10
           5       0.20      0.80      0.31        10
           6       0.00      0.00      0.00        10
           

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
rpt_cnn_clsfctn=5
rslt=np.array([])
rslts=np.array([])
xtrain=np.empty((0,len(wndws1[0])), float) 
rprt=np.empty((0,3),float)


for i in range (1,cls_num+1):                           #تعریف آرایه ی پنجره های کلاس ها
    vars()['ecg_aug10'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug11'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug12'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug20'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug21'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug22'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug30'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug31'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug32'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug40'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug41'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug42'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug50'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug51'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 
    vars()['ecg_aug52'+str(i)]=xtrain=np.empty((0,smpl_rte+1), float) 

for i in range (1,cls_num+1):                                     #تعریف آرایه ی پنجره های کلاس ها
    vlum_win=mx_wndws-len(vars()['wndws'+str(i)])         #میزان داده افزایی در روش های عمیق (پنجره)
    #print("vlum_win=",vlum_win)
    vars()['ecg_aug10'+str(i)]=np.append(vars()['ecg_aug10'+str(i)],sclRaw(ecg,i,vlum_win,mag=.05),axis=0)
    vars()['ecg_aug11'+str(i)]=np.append(vars()['ecg_aug11'+str(i)],sclRaw(ecg,i,vlum_win,mag=.1),axis=0)
    vars()['ecg_aug12'+str(i)]=np.append(vars()['ecg_aug12'+str(i)],sclRaw(ecg,i,vlum_win,mag=.2),axis=0)
    vars()['ecg_aug20'+str(i)]=np.append(vars()['ecg_aug20'+str(i)],MgWrpRaw(ecg,i,vlum_win,mag=.05),axis=0)
    vars()['ecg_aug21'+str(i)]=np.append(vars()['ecg_aug21'+str(i)],MgWrpRaw(ecg,i,vlum_win,mag=.1),axis=0)
    vars()['ecg_aug22'+str(i)]=np.append(vars()['ecg_aug22'+str(i)],MgWrpRaw(ecg,i,vlum_win,mag=.2),axis=0)
    vars()['ecg_aug30'+str(i)]=np.append(vars()['ecg_aug30'+str(i)],TimWrpRaw(ecg,i,vlum_win,mag=.05),axis=0)
    vars()['ecg_aug31'+str(i)]=np.append(vars()['ecg_aug31'+str(i)],TimWrpRaw(ecg,i,vlum_win,mag=.1),axis=0)
    vars()['ecg_aug32'+str(i)]=np.append(vars()['ecg_aug32'+str(i)],TimWrpRaw(ecg,i,vlum_win,mag=.2),axis=0)
    vars()['ecg_aug40'+str(i)]=np.append(vars()['ecg_aug40'+str(i)],GDORaw(ecg,i,vlum_win,mag=.05),axis=0)
    vars()['ecg_aug41'+str(i)]=np.append(vars()['ecg_aug41'+str(i)],GDORaw(ecg,i,vlum_win,mag=.1),axis=0)
    vars()['ecg_aug42'+str(i)]=np.append(vars()['ecg_aug42'+str(i)],GDORaw(ecg,i,vlum_win,mag=.2),axis=0)
    vars()['ecg_aug50'+str(i)]=np.append(vars()['ecg_aug50'+str(i)],frqncRaw(furir_windws,i,vlum_win,mag=.05),axis=0)
    vars()['ecg_aug51'+str(i)]=np.append(vars()['ecg_aug51'+str(i)],frqncRaw(furir_windws,i,vlum_win,mag=.1),axis=0)
    vars()['ecg_aug52'+str(i)]=np.append(vars()['ecg_aug52'+str(i)],frqncRaw(furir_windws,i,vlum_win,mag=.2),axis=0)    

for methd in range(1,6):
    for Mgntud in range(0,3):
        xtrain=np.empty((0,len(wndws1[0])), float) 
        rprt=np.empty((0,3),float)
        for repeat in range(1,rpt_cnn_clsfctn+1):
            for i in range (1,cls_num+1):   
                xtrain=np.append(xtrain,vars()['wndws'+str(i)],axis=0)
                xtrain=np.append(xtrain,vars()['ecg_aug'+str(methd)+str(Mgntud)+str(i)],axis=0)
                #print('Up to class ', i, 'train shape = ', np.shape(xtrain))

            # Normalization train windows
            mx_aug=np.max(xtrain[:,:-1])
            mx=np.max(ecg[:,:-1])
            mn_aug=np.min(xtrain[:,:-1])
            mn=np.min(ecg[:,:-1])
            xtrain[:,:-1]= 2*(xtrain[:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1


            ###Test Windows:
            wndws_test=np.empty((0,len(wndws1[0])), float)
            i=0
            windws=np.array([])
            for cls in range (1,cls_num+1):                                 #ساخت پنجره های داده های آموزش اصلی
                vars()['wndws_tst'+str(cls)]=np.empty((0,len(wndws1[0])), float)
                vars()['wndws_tst'+str(cls)]=np.append(vars()['wndws_tst'+str(cls)],vars()['ecg_tst'+str(cls)],axis=0)   

            # Normalization test windows
            for i in range (1,cls_num+1):                
                cls=i
                #print('\n cls', i, ' >> ')
                #print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
                #print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))
                vars()['wndws_tst'+str(i)][:,:-1]= 2*(vars()['wndws_tst'+str(i)][:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1
                #print('after normalizing >>')
                #print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
                #print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))
            #print("each class and its windows =")                    #تجمیع کلاس های تست
            cls_wndws=np.array([])
            for cls in range (1,cls_num+1):                                # آرایه ی تعداد پنجره ی هر کلاس
                wns=len(vars()['wndws_tst'+str(cls)])
                cls_wndws=np.append(cls_wndws,wns)
                #print(cls, wns, end='\t')
                wndws_test=np.append(wndws_test,vars()['wndws_tst'+str(cls)],axis=0)
            #mx_wndws=int(np.max(cls_wndws))                        # حداکثر تعداد پنجره ی موجود بین کلاس ها
            #print('\n max instance in classes of test windows =', mx_wndws)

            #print('min tst =', np.min(wndws_test[:,:-1]))
            #print('max tst =', np.max(wndws_test[:,:-1]))

            Xtest=wndws_test[:,:-1]
            ytest=np.int16(wndws_test[:,-1])

            #fullprint(ytest)

            trainx=np.random.permutation(xtrain)
            Xtrain=np.array(trainx[:,:-1])
            ytrain=np.int16(trainx[:,-1])

            X_train=np.array(Xtrain)
            y_train=np.array(ytrain)
            X_test=np.array(Xtest)
            y_test=np.array(ytest)
            X_valid=np.array(Xtrain)
            y_valid=np.array(ytrain)
            #print('X_train => ', X_train.shape)
            #print('y_train => ', y_train.shape)
            #print('X_test  => ', X_test.shape)
            #print('y_test  => ', y_test.shape)
            #print('X_valid  => ', X_valid.shape)
            #print('y_valid  => ', y_valid.shape)

            y_train_cat=to_categorical(y_train)
            y_valid_cat=to_categorical(y_valid)
            y_test_cat=to_categorical(y_test)
            X_train = np.expand_dims(X_train, axis=2)
            X_valid = np.expand_dims(X_valid, axis=2)
            X_test = np.expand_dims(X_test, axis=2)

            ####### NETWORK #######
            #Yildirin

            accuracy=0
            acc_crs=np.array([])
            ##for i in range (10):
            ##    if accuracy<0.2 :

            verbose, epochs, batch_size = 0, 1000, btch
            n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train_cat.shape[1]
            steps_per_epoch = len(X_train)//batch_size
            validation_steps = len(X_valid)//batch_size # if you have test data



            model_crs = Sequential()
            #model_crs.add(Conv1D(input_shape=x_train.shape[1:],filters=64,kernel_size=9,padding="same", activation="relu"))
            model_crs.add(Conv1D(filters=256, kernel_size=12, strides=1, activation='relu', input_shape=(n_timesteps,n_features))) #Replaced
            model_crs.add(BatchNormalization())
            model_crs.add(MaxPooling1D(pool_size=2,strides=3))

            model_crs.add(Conv1D(filters=64, kernel_size=7, strides=1, padding="same", activation="relu"))
            model_crs.add(BatchNormalization())
            model_crs.add(MaxPooling1D(pool_size=2,strides=2))

            model_crs.add(Conv1D(filters=64, kernel_size=10, strides=1, padding="same", activation="relu"))
            model_crs.add(Conv1D(filters=256, kernel_size=5, strides=2, padding="same", activation="relu"))
            model_crs.add(MaxPooling1D(pool_size=2,strides=2))

            model_crs.add(Conv1D(filters=512, kernel_size=15, strides=1, padding="same", activation="relu"))
            model_crs.add(MaxPooling1D(pool_size=2,strides=2))

            model_crs.add(Conv1D(filters=1024, kernel_size=5, strides=1, padding="same", activation="relu"))
            #model_crs.add(Conv1D(filters=128, kernel_size=3, strides=1, padding="same", activation="relu"))


            model_crs.add(Flatten())
            model_crs.add(Dropout(0.1))
            model_crs.add(Dense(units=1024,activation="relu"))
            #model_crs.add(Dense(units=2048,activation="relu"))
            model_crs.add(Dense(n_outputs, activation='softmax'))   #Replaced here from old Model

            model_crs.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
            # fit network   #CategoricalCrossentropy #sparse_categorical_crossentropy #SparseCategoricalCrossentropy

            earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", mode ="max", patience = 50, restore_best_weights = True)

            model_crs.fit(X_train, y_train_cat, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data = (X_valid, y_valid_cat), callbacks =[earlystopping])

            _, accuracy = model_crs.evaluate(X_valid, y_valid_cat, batch_size=batch_size, verbose=verbose)


            predict_x=model_crs.predict(X_test)              # Function 1
            y_pred_crs=maxindx(predict_x)                    # function from augment.py to remove 0 index predictions

            rprt0=classification_report(y_test, y_pred_crs,output_dict=True)
            rprt_row=np.array([])
            rprt_row=np.append(rprt_row,rprt0['accuracy'])
            rprt_row=np.append(rprt_row,rprt0['macro avg']['f1-score'])
            rprt_row=np.append(rprt_row,rprt0['weighted avg']['f1-score'])
            rprt=np.append(rprt,np.array([rprt_row]),axis=0)
        maxm=rprt[np.argmax(rprt[:,1])]
        avrg=np.mean(rprt,axis=0)
        rslts=np.append(rslts,['mthd '+str(methd)+'  Mgntud '+str(Mgntud)+'  average= '+str(np.int16(100*avrg))+'  max= '+str(np.int16(100*maxm))+'  var= '+str(np.int16(np.var(100*rprt[:,1])))],axis=0)

In [ ]:
predict_x=model_crs.predict(X_test)              # Function 1

y_pred_crs=maxindx(predict_x)                    # function from augment.py to remove 0 index predictions

#y_pred_crs = model_crs.predict_classes(X_test)

cm = confusion_matrix(y_test, y_pred_crs)
fullprint(cm)
acc3=accuracy_score(y_test, y_pred_crs)

print(classification_report(y_test, y_pred_crs))

#sns.heatmap(cm, annot=True)

#plt.imshow(cm)